In [1]:
import snscrape.modules.twitter as sntwitter
import csv
import pandas as pd
import time
import re  
from wordcloud import WordCloud# Join the different processed titles together.
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter("ignore", DeprecationWarning)# Load the LDA model from sk-learn
from sklearn.decomposition import LatentDirichletAllocation as LDA
import os
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import gensim
from gensim import corpora, models, similarities
import logging
import tempfile
from nltk.corpus import stopwords
from string import punctuation
from collections import OrderedDict
import pyLDAvis.gensim
import tempfile
import nltk
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from IPython.display import display
from apyori import apriori
import spacy
from vaderSentiment import vaderSentiment
from datetime import datetime, timedelta
import pandas_bokeh

%matplotlib inline

init_notebook_mode(connected=True) #do not miss this line
warnings.filterwarnings("ignore")

# Search Query

In [2]:
#############################################################
#############################################################
##                                                         ##
##  INPUT SEARCH QUERY TO SCRAPE TWEETS AND NAME CSV FILE  ##
##                                                         ##
#############################################################
#############################################################


keyword = '@mittromney'

maxTweets = 5000


#Open/create a file to append data to
dt = datetime.now().strftime("%d%b%Y_%H%M") # ("%d_%b_%Y_%H_%M_%S")  --  add %S for seconds
csvFile = open('twitter_data/twitter_{}_{}_{}.csv'.format(keyword, maxTweets, dt), 'a', newline='', encoding='utf8')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['id','date','tweet']) 

for i,tweet in enumerate(sntwitter.TwitterSearchScraper(keyword).get_items()) :
        if i > maxTweets :
            break      
        csvWriter.writerow([tweet.id, tweet.date, tweet.content])
csvFile.close()

In [3]:
##################################################
# SPECIFY ADDITIONAL WORDS AND ARTEFACTS TO DELETE
##################################################

delete_words = ["amp", "https", "http", "n't", "'s", "'m", "www", "nan"]

# Load data

In [4]:
filename = 'twitter_data/twitter_{}_{}_{}.csv'.format(keyword, maxTweets, dt)

In [5]:
# filename = 'gggggggg.csv'

# EDA

In [6]:
pd.set_option('max_colwidth',150)
tweets = pd.read_csv(filename)
tweets.columns = ['ID', 'Datetime', 'Text']
# Convert tweets to strings and lower case
tweets['Text'] = tweets['Text'].astype(str)
tweets['Text'] = tweets['Text'].map(lambda x: x.lower())
print("Total tweets:   ", len(tweets))
print("Start datetime (UTC): ", tweets.Datetime.min())
print("End datetime (UTC):   ", tweets.Datetime.max())

# tweets

Total tweets:    5001
Start datetime (UTC):  2016-02-17 10:44:39+00:00
End datetime (UTC):    2020-11-20 13:58:57+00:00


# Filter data by date

In [7]:
datetime.utcnow()

datetime.datetime(2020, 11, 20, 14, 0, 1, 774852)

In [8]:
# yesterday_datetime = datetime.utcnow() - timedelta(2)  # datetime 24 hours earlier (1)
# yesterday_datetime = yesterday_datetime.strftime("%Y-%m-%d %H:%M:%S")
# tweets = tweets.query('Datetime >= "{}+00:00"'.format(yesterday_datetime))
# print("Starting datetime: ", tweets.Datetime.min())
# print("Total tweets:      ", len(tweets))

# Identify duplicated tweets

In [9]:
duplicated_tweets = tweets.loc[tweets.duplicated(subset = ['Text'], keep = 'first')]
unique_duplicated_tweets = pd.DataFrame(duplicated_tweets['Text']).drop_duplicates(subset = 'Text', keep = 'first')
print("Total duplicated Tweets: ", len(duplicated_tweets))
print("Total unique duplicated Tweets: ", len(unique_duplicated_tweets))

Total duplicated Tweets:  123
Total unique duplicated Tweets:  31


In [10]:
# count duplicated tweets using groupby
dup_tweets = tweets.groupby(tweets.Text.tolist()).size().reset_index().rename(columns={0:'frequency'})
dup_tweets = dup_tweets.query('frequency > 1').sort_values(by ='frequency', ascending = False, inplace = False)
pd.set_option('display.max_rows', len(dup_tweets))

In [11]:
pandas_bokeh.output_notebook()

if len(dup_tweets) > 0:
    dup_tweets.plot_bokeh(kind = 'line',
                      x = 'index',
                      y = 'frequency', 
                      xlabel = 'Tweet',
                      ylabel = 'Frequency',
                      title = 'Number of retweets',
                      figsize = (800, 400),
                      xticks = "")
else:
    pass

Loading BokehJS ...

In [12]:
dup_tweets

,index,frequency
4822,thousands of happy families from australia have already taken advantage of it.\nhttps://t.co/f3mnjiedsr https://t.co/gyfud70w5q,45
2969,@mittromney thank you.,16
4642,"immediately after henry viii saw a portrait of anne of cleves, he made her his bride. but when they finally met, his blood ran cold: she was hideo...",13
2798,@mittromney thank you,7
2936,@mittromney thank you!,6
3163,@mittromney traitor,5
4559,at https://t.co/hpy1rhyip7 we believe #contextmatters. knowing big from small brings perspective #numberscount #math https://t.co/wpfvxvft0c,4
1770,@mittromney do something.,4
3251,@mittromney what are you going to do about it?,3
2940,@mittromney thank you!!,3


In [13]:
index_num = 978

if index_num in dup_tweets.index: display(dup_tweets['index'].loc[index_num])

# Drop duplicates

In [14]:
# Remove duplicate rows
# tweets.drop_duplicates(subset = ['Text'], keep = 'first', inplace = True)
print("Total tweets without duplicates: ", len(tweets))

Total tweets without duplicates:  5001


# Drop rows containing keywords

In [15]:
remove_SPAM = pd.DataFrame()
remove_SPAM = tweets[tweets['Text'].str.contains("hhhhhhhhhhhhh")] 

remove_SPAM_index = remove_SPAM.index.tolist()
tweets.drop(remove_SPAM_index, inplace = True)
tweets.reset_index()
tweets_original = tweets  # for search function below
print("Total tidied replies: ", len(tweets))

Total tidied replies:  5001


# Extract hashtags

In [16]:
def search_hashtag(string):
    pat = r'(\w*%s\w*)' % "#"      # find strings containing '#'
    return re.findall(pat, string)

hashtags = pd.DataFrame()  # add hashtags to new dataframe
hashtags['list'] = tweets['Text'].apply(lambda row: search_hashtag(row))  
hashtags_tokenised = pd.DataFrame()  
hashtags_tokenised['hashtags'] = hashtags.apply(lambda x: pd.Series(x['list']),axis=1).stack().reset_index(level=1, drop=True)
total_number_hashtags = len(hashtags_tokenised.hashtags.unique())
print("Total number of unique hashtags: ", total_number_hashtags)

Total number of unique hashtags:  340


# Identify top hashtags

In [17]:
hash_list = hashtags_tokenised['hashtags'].str.split(expand=True).stack().value_counts()
top_hashtags = pd.DataFrame(hash_list).reset_index()
top_hashtags.columns = ['hashtag', 'frequency']

In [18]:
pandas_bokeh.output_notebook()

top_hashtags.plot_bokeh(kind = 'line',
                      x = 'hashtag',
                      y = 'frequency', 
                      xlabel = 'Hashtag',
                      ylabel = 'Frequency',
                      title = 'Top hashtags',
                      figsize = (800, 400),
                      xticks = "")

Loading BokehJS ...

Figure(id='1093', ...)

In [19]:
pd.set_option('display.max_rows', len(top_hashtags))

top_hashtags_styler = top_hashtags.style.set_properties(**{'text-align': 'left'})
top_hashtags_styler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

,hashtag,frequency
0,#gop,13
1,#countryoverparty,12
2,#coup,12
3,#trump,10
4,#rino,9
5,#stopthesteai,8
6,#25thamendmentnow,7
7,#trumpisanationaldisgrace,7
8,#gopcorruptionovercountry,7
9,#gopcomplicittraitors,6


# Define apriori association rules

In [20]:
# Subset data for more manageable apriori analysis

# hashtags_subset = hashtags['list'].sample(n = 2500)

In [21]:
# Swap hashtags['list'] with hashtags_subset if apriori becomes too computationally complex
hashtags_list = hashtags['list'].tolist()
association_rules = apriori(hashtags_list, min_support=0.1, min_confidence=0.1, min_lift=1, min_length=2)
association_results = list(association_rules)
len(association_results)

0

In [22]:
# association_results[0]

In [23]:
# Convert rules to dataframe

df = pd.DataFrame(columns=('Items','Antecedent','Consequent','Support','Confidence','Lift'))

Support =[]
Confidence = []
Lift = []
Items = []
Antecedent = []
Consequent=[]

for RelationRecord in association_results:
    for ordered_stat in RelationRecord.ordered_statistics:
        Support.append(RelationRecord.support)
        Items.append(RelationRecord.items)
        Antecedent.append(ordered_stat.items_base)
        Consequent.append(ordered_stat.items_add)
        Confidence.append(ordered_stat.confidence)
        Lift.append(ordered_stat.lift)

df['Items'] = Items                          
df['Antecedent'] = Antecedent
df['Consequent'] = Consequent
df['Support'] = Support
df['Confidence'] = Confidence
df['Lift']= Lift

### Association results

In [24]:
pd.set_option('display.max_rows', df.shape[0]+1)

df.sort_values(by ='Support', ascending = False, inplace = False)

,Items,Antecedent,Consequent,Support,Confidence,Lift


### Unique Item Combinations

In [25]:
df_unique_hashtag_items = df.drop_duplicates(subset = ['Items'], keep = 'first', inplace = False)
df_unique_hashtag_items.sort_values(by ='Support', ascending = False, inplace = True)

df_unique_hashtag_items[['Items', 'Support']]

,Items,Support


# Identify Apriori rows containing specific hashtag

In [26]:
#############
# ITEM SEARCH
#############


ITEM_SEARCH = '#demsgotoprison'


df_search = df[df.Items.apply(lambda x: ITEM_SEARCH in x)]
print("Hashtag " + ITEM_SEARCH +" occurrences: ", len(df_search))
df_search.head(50)

Hashtag #demsgotoprison occurrences:  0


""


# Identify top mentions

In [27]:
def search_mentions(string):
    pat = r'(\w*%s\w*)' % "@"      
    return re.findall(pat, string)

mentions = pd.DataFrame()  # add hashtags to new dataframe
mentions['list'] = tweets['Text'].apply(lambda row: search_mentions(row))  
mentions_tokenised = pd.DataFrame()  
mentions_tokenised['mentions'] = mentions.apply(lambda x: pd.Series(x['list']),axis=1).stack().reset_index(level=1, drop=True)
total_number_mentions = len(mentions_tokenised.mentions.unique())
print("Total number of unique mentions: ", total_number_mentions)
mentions_list = mentions_tokenised['mentions'].str.split(expand=True).stack().value_counts()

Total number of unique mentions:  1247


In [28]:
pd.set_option('display.max_rows', len(mentions_list))
top_mentions = pd.DataFrame(mentions_list).reset_index()
top_mentions.columns = ['mention', 'frequency']

In [29]:
pandas_bokeh.output_notebook()

top_mentions.plot_bokeh(kind = 'line',
                        x = 'mention',
                        y = 'frequency', 
                        xlabel = 'Mentions',
                        ylabel = 'Frequency',
                        title = 'Top mentions',
                        figsize = (800, 400),
                        xticks = "")

Loading BokehJS ...

Figure(id='1187', ...)

In [30]:
top_mentions

top_mentions_styler = top_mentions.style.set_properties(**{'text-align': 'left'})
top_mentions_styler.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])

,mention,frequency
0,@mittromney,5012
1,@ingrahamangle,248
2,@realdonaldtrump,238
3,@tribelaw,184
4,@gop,175
5,@bensasse,146
6,@neil10790465,103
7,@senatemajldr,90
8,@marcorubio,80
9,@maxboot,74


# Extract nouns, verbs and adjectives

In [ ]:
lemmatizer = nltk.WordNetLemmatizer()

def leaves(tree):
    """Finds NP (nounphrase) leaf nodes of a chunk tree."""
    for subtree in tree.subtrees(filter = lambda t: t.label() =='NP'):
        yield subtree.leaves()       
def get_word_postag(word):
    if pos_tag([word])[0][1].startswith('J'):
        return wordnet.ADJ
    if pos_tag([word])[0][1].startswith('V'):
        return wordnet.VERB
    if pos_tag([word])[0][1].startswith('N'):
        return wordnet.NOUN
    else:
        return wordnet.NOUN  
def normalise(word):
    """Normalises words to lowercase and stems and lemmatizes it."""
    word = word.lower()
    postag = get_word_postag(word)
    word = lemmatizer.lemmatize(word,postag)
    return word
def get_terms(tree):    
    for leaf in leaves(tree):
        terms = [normalise(w) for w,t in leaf]
        yield terms  
        
tidied_tweets = []        
for t in tweets['Text']:
    #word tokenizeing and part-of-speech tagger
    document = t
    tokens = [nltk.word_tokenize(sent) for sent in [document]]
    postag = [nltk.pos_tag(sent) for sent in tokens][0]    
    # Rule for NP chunk and VB Chunk
    grammar = r"""
        NBAR:
            {<NN.*|JJ>*<NN.*>}  # Nouns and Adjectives, terminated with Nouns
            {<RB.?>*<VB.?>*<JJ>*<VB.?>+<VB>?} # Verbs and Verb Phrases            
        NP:
            {<NBAR>}
            {<NBAR><IN><NBAR>}  # Above, connected with in/of/etc...           
    """
    #Chunking
    cp = nltk.RegexpParser(grammar)    
    # the result is a tree
    tree = cp.parse(postag)    
    terms = get_terms(tree)    
    features = []
    for term in terms:
        _term = ''
        for word in term:
            _term += ' ' + word
        features.append(_term.strip())      
    tidied_tweets.append(features) 
tweet_list = [[] for i in range(len(tidied_tweets))]
for tweets, t in zip(tidied_tweets, range(len(tidied_tweets))):
    for words in tweets:
        a = re.findall(r"[\w']+", words)  ## ADD JUST THIS TO THE NEW CODE AND TIDY OUTPUT
        tweet_list[t].append(a)       
# Remove empty lists from tweet_list if delete_words leaves anything empty
tweet_list = [x for x in tweet_list if x != []]
        
# Un-nest inner most list (individual words)
from functools import reduce
single_list = []
for t in tweet_list:
    a = reduce(lambda x,y: x+y, t)
    single_list.append(a)  
single_list = [[y for y in x if y not in delete_words] for x in single_list]
# Converted child list to string in new 'tidied_list'
tidied_list = []
separator = " "
for l in single_list:
    a = separator.join(l)
    tidied_list.append(a)  
tweets = pd.DataFrame(tidied_list, columns = ['Text_processed'])

# Exploratory Analysis

In [ ]:
long_string = ','.join(list(tweets['Text_processed'].values))# Create a WordCloud object
wordcloud = WordCloud(background_color="white", max_words=100, contour_width=3, contour_color='steelblue')# Generate a word cloud
wordcloud.generate(long_string)# Visualize the word cloud
wordcloud.to_image()

# Prepare text for LDA analysis

In [ ]:
sns.set_style('whitegrid')
%matplotlib inline  
def plot_30_most_common_words(count_data, count_vectorizer):
    import matplotlib.pyplot as plt
    words = count_vectorizer.get_feature_names()
    total_counts = np.zeros(len(words))
    for t in count_data:
        total_counts+=t.toarray()[0]   
    count_dict = (zip(words, total_counts))
    count_dict = sorted(count_dict, key=lambda x:x[1], reverse=True)[0:30]
    words = [w[0] for w in count_dict]
    counts = [w[1] for w in count_dict]
    x_pos = np.arange(len(words)) 
    
    plt.figure(2, figsize=(15, 15/1.6180))
    plt.subplot(title='30 most common words')
    sns.set_context("notebook", font_scale=3, rc={"lines.linewidth": 2.5})
    sns.barplot(x_pos, counts, color='crimson')
    plt.xticks(x_pos, words, rotation=90) 
    plt.ylabel('Frequency')
    plt.show()# Initialise the count vectorizer with the English stop words   
count_vectorizer = CountVectorizer(stop_words='english')# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(tweets['Text_processed']) # Visualise the 30 most common words
plot_30_most_common_words(count_data, count_vectorizer)

In [ ]:
####################
####################
##  TOTAL TOPICS  ##
####################
####################


total_topics = int(input("Total topics: "))

# LDA model training and results visualization

In [ ]:
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))       
# Tweak the two parameters below
number_words = 20  # Create and fit the LDA model
lda = LDA(n_components=total_topics, n_jobs=-1)
lda.fit(count_data)

# print("Topics found via LDA:")
# print_topics(lda, count_vectorizer, number_words)

# Using pyLDAvis with twitter data

In [ ]:
# Preparing a corpus for analysis and checking first 5 entries
corpus=[]
a=[]
for i in range(len(tweets['Text_processed'])):
        a=tweets['Text_processed'][i]
        corpus.append(a)  
        
# corpus[0:5]

In [ ]:
# create temp folder
TEMP_FOLDER = tempfile.gettempdir()
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# removing common words and tokenizing
list1 = ['RT','rt']
stoplist = stopwords.words('english') + list(punctuation) + list1
texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]  # reused in paragraph search below
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold

In [ ]:
# Create ordered dictionary
data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T
# print(df_lda.shape)

In [ ]:
sns.set(font_scale=1.4)
g=sns.clustermap(df_lda.corr(), center=0, standard_scale=1, cmap="RdBu", metric='cosine', linewidths=.75, figsize=(20, 20))
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.show()
#plt.setp(ax_heatmap.get_yticklabels(), rotation=0)  # For y axis

#### Show first n important words in the topics

In [ ]:
#Show first n important word in the topics:
lda_list = lda.show_topics(total_topics, 50)
topic_df = pd.DataFrame()
for t in range(total_topics):
    lda_list_n = lda_list[t][1].split(" + ")  # split includes a space around the '+'
    topic_df["Word " + str(t + 1)] = [i.split('*')[1] for i in lda_list_n]
    topic_df["Score " + str(t + 1)] = [i.split('*')[0] for i in lda_list_n]  
    
# topic_df

In [ ]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel

# Phrase search

In [ ]:
###############
# PHRASE SEARCH
###############

search_word = "sidneypowell1"

subset_search_word = ""


def analyse_corpus_sentiment(text):
    english = spacy.load("en_core_web_sm") 
    negative = []
    neutral = []
    positive = []
    compound = []
    for t in text:
        result = english(t)
        sentences = [str(s) for s in result.sents]
        analyzer = vaderSentiment.SentimentIntensityAnalyzer()
        sentiment = [analyzer.polarity_scores(str(s)) for s in sentences]
        negative.append(sentiment[0]['neg'])
        neutral.append(sentiment[0]['neu'])
        positive.append(sentiment[0]['pos'])
        compound.append(sentiment[0]['compound'])
    sentiment_score = pd.DataFrame()
    sentiment_score['Negative'] = [round(np.mean(negative), 3)]
    sentiment_score['Neutral'] = [round(np.mean(neutral), 3)]
    sentiment_score['Positive'] = [round(np.mean(positive), 3)]
    sentiment_score['Compound'] = [round(np.mean(compound), 3)]
    display(sentiment_score)

text_dat = tweets_original['Text']
comments_lower = pd.DataFrame()
comments_lower['comment'] = text_dat.apply(lambda row: row.lower())  
searched_comment = comments_lower[(comments_lower['comment'].str.contains(search_word)) & 
                                  (comments_lower['comment'].str.contains(subset_search_word))] 
searched_comment_index = searched_comment.index.tolist()
sci = pd.DataFrame()
sci['index'] = searched_comment_index
sci.set_index('index', inplace = True)
searched_comment_joined = pd.merge(sci, text_dat, left_on = 'index', right_index = True)
searched_comments_tidied = searched_comment_joined.Text.map(lambda x: re.sub('[“”]', "", x))  # Remove quotation marks (“”) from text
pd.set_option('max_colwidth',600)
if len(searched_comment_joined) <= 50:
    print("Searched comment length: ", len(searched_comment_joined))
    analyse_corpus_sentiment(searched_comments_tidied)
    display(searched_comment_joined.head(50))
else:
    print("Searched comment length: ", len(searched_comment_joined))
    analyse_corpus_sentiment(searched_comments_tidied)
    display(searched_comment_joined.sample(n = 50).sort_values(by = 'index', ascending = True, inplace = False))

In [ ]:
index_num = 152

if index_num in searched_comment_joined.index: display(searched_comment_joined.Text.loc[index_num])